<a href="https://colab.research.google.com/github/jrojaslegua/practicante-habitat-test/blob/applicants/Examen%20AFP%20HABITAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from datetime import datetime
from sqlalchemy import create_engine, Column, Integer, String, DateTime, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship

# Creamos la base de datos y la sesión
Base = declarative_base()
engine = create_engine('sqlite:///estacionamiento.db')
Session = sessionmaker(bind=engine)

# Definimos las clases utilizando SQLAlchemy

class Vehiculo(Base):
    __tablename__ = 'vehiculos'

    id = Column(Integer, primary_key=True)
    numero_placa = Column(String, unique=True, nullable=False)
    tipo_vehiculo = Column(String, nullable=False)
    estancias = relationship('Estancia', back_populates='vehiculo')

class Estancia(Base):
    __tablename__ = 'estancias'

    id = Column(Integer, primary_key=True)
    vehiculo_id = Column(Integer, ForeignKey('vehiculos.id'), nullable=False)
    hora_entrada = Column(DateTime, nullable=False)
    hora_salida = Column(DateTime)

    vehiculo = relationship('Vehiculo', back_populates='estancias')

# Creamos las tablas en la base de datos
Base.metadata.create_all(engine)

# Definimos las clases y funciones restantes

class EmpleadoEstacionamiento:
    def __init__(self):
        self.session = Session()

    def registrar_entrada_vehiculo(self, vehiculo, hora_entrada):
        nueva_estancia = Estancia(vehiculo=vehiculo, hora_entrada=hora_entrada)
        self.session.add(nueva_estancia)
        self.session.commit()
        print(f"Vehículo con placa {vehiculo.numero_placa} ha entrado al estacionamiento a las {hora_entrada.strftime('%H:%M:%S')}")

    def registrar_salida_vehiculo(self, vehiculo, hora_salida):
        estancia = self.session.query(Estancia).filter_by(vehiculo=vehiculo, hora_salida=None).first()
        if estancia:
            estancia.hora_salida = hora_salida
            self.session.commit()
            print(f"Vehículo con placa {vehiculo.numero_placa} ha salido del estacionamiento a las {hora_salida.strftime('%H:%M:%S')}")

            if vehiculo.tipo_vehiculo == 'oficial':
                print("Estancia registrada para vehículo oficial.")
            elif vehiculo.tipo_vehiculo == 'residente':
                tiempo_estancia = (hora_salida - estancia.hora_entrada).seconds // 60
                vehiculo.tiempo_acumulado += tiempo_estancia
                print(f"Tiempo estacionado por vehículo residente: {vehiculo.tiempo_acumulado} minutos.")
            else:
                tarifa = Tarifa()
                importe = tarifa.calcular_importe(estancia)
                print(f"Importe a pagar por vehículo no residente: MXN {importe:.2f}")
        else:
            print("El vehículo no está registrado.")

class Tarifa:
    def calcular_importe(self, estancia):
        tiempo_estancia = (estancia.hora_salida - estancia.hora_entrada).seconds // 60
        return tiempo_estancia * 0.5

# Función principal para simular la interacción con el empleado

if __name__ == "__main__":
    main()

<ipython-input-8-327b45c3194d>:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


NameError: ignored